In [1]:
# pip install utils

In [2]:
# pip install geopy

In [ ]:
# pip install fuzzywuzzy

In [1]:
# Standard things you should have in a Python distro
import os
import requests
import utils
from datetime import datetime
import time

In [2]:
# Run of the mill Data Science packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Project specific imports
import geopandas as gpd
from geopandas.tools import geocode
from geopy.geocoders import Nominatim

In [4]:
# needed for getting GeoData
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [5]:
# Import weather data for the city of San Francisco.
weather_data = pd.read_csv('weather.csv')
weather_data.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,8/30/2013,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,8/31/2013,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,9/1/2013,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,9/2/2013,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


In [6]:
# Covert the date column of weather into a DateTime object.
weather_data['date'] = pd.to_datetime(weather_data['date'])

In [7]:
# Check all unique events
weather_data['events'].unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'],
      dtype=object)

In [8]:
# Set events with None values to "clear", as in clear days. Then, convert events to lower case.
clear_days_row_indexes=list(weather_data[weather_data['events'].isna()].index.values)
for row in clear_days_row_indexes:
    weather_data.loc[row, 'events'] = 'clear'
weather_data['events'] = weather_data['events'].apply(lambda x: x.lower())

# Check that the substitution took place
weather_data['events'].unique()

array(['clear', 'fog', 'rain', 'fog-rain', 'rain-thunderstorm'],
      dtype=object)

In [9]:
# Check the range for cloud_cover
weather_data['cloud_cover'].unique()

array([ 4.,  2.,  6.,  3.,  0.,  1.,  5.,  7.,  8., nan])

In [10]:
# Check how many nan values occur in cloud_cover
weather_data['cloud_cover'].isnull().sum()

1

In [11]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            3665 non-null   datetime64[ns]
 1   max_temperature_f               3661 non-null   float64       
 2   mean_temperature_f              3661 non-null   float64       
 3   min_temperature_f               3661 non-null   float64       
 4   max_dew_point_f                 3611 non-null   float64       
 5   mean_dew_point_f                3611 non-null   float64       
 6   min_dew_point_f                 3611 non-null   float64       
 7   max_humidity                    3611 non-null   float64       
 8   mean_humidity                   3611 non-null   float64       
 9   min_humidity                    3611 non-null   float64       
 10  max_sea_level_pressure_inches   3664 non-null   float64       
 11  mean

In [12]:
# Check how many nan values occur in max_gust_speed_mph
weather_data['max_gust_speed_mph'].isnull().sum()

899

In [13]:
# Drop the max_gust_speed_mph column
weather_data = weather_data.drop('max_gust_speed_mph', axis=1)

In [14]:
# Check the range for precipitation_inches
weather_data['precipitation_inches'].unique()

array(['0', '0.23', 'T', '0.01', '0.28', '0.63', '0.29', '0.06', '0.85',
       '0.09', '0.64', '0.42', '0.35', '0.43', '0.22', '0.74', '0.03',
       '0.12', '0.16', '0.49', '0.17', '0.08', '0.04', '0.53', '0.07',
       '0.02', '0.83', '1.06', '1.71', '0.37', '0.27', '0.45', '0.78',
       '0.88', '0.66', '0.47', '0.1', '0.61', '0.14', '0.05', '0.68',
       '0.97', '0.26', '0.15', '0.87', '0.57', '0.69', '0.32', '0.21',
       '0.24', '0.52', '0.36', '0.33', '0.25', '0.11', '0.2', '1.18',
       '1.43', '3.12', '0.48', '0.19', '1.09', '0.65', '0.13', '0.91',
       '0.99', '0.18', '0.4', '1.07', nan, '0.41', '0.34', '1.25', '1.85',
       '3.36', '0.71', '1.3', '0.72', '0.6', '0.51', '1.2', '1.28',
       '3.23', '0.55', '1.26', '0.39'], dtype=object)

In [15]:
# Check if weather_data contains data from more than one weather station
weather_data.sort_values('date')

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,2013-08-29,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,10.0,23.0,11.0,0,4.0,clear,286.0,94107
552,2013-08-29,80.0,70.0,64.0,65.0,61.0,58.0,83.0,72.0,55.0,...,10.0,10.0,10.0,16.0,5.0,0,4.0,clear,355.0,94041
368,2013-08-29,78.0,71.0,64.0,62.0,61.0,60.0,87.0,71.0,54.0,...,10.0,10.0,10.0,20.0,8.0,0,4.0,clear,355.0,94301
184,2013-08-29,80.0,71.0,62.0,63.0,59.0,57.0,94.0,79.0,48.0,...,10.0,10.0,10.0,14.0,6.0,0,5.0,clear,313.0,94063
736,2013-08-29,81.0,72.0,63.0,62.0,61.0,59.0,87.0,69.0,51.0,...,10.0,10.0,10.0,16.0,7.0,0,4.0,clear,320.0,95113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3299,2015-08-31,82.0,72.0,61.0,62.0,56.0,52.0,84.0,63.0,42.0,...,10.0,10.0,10.0,22.0,6.0,0,0.0,clear,6.0,94041
2569,2015-08-31,80.0,71.0,62.0,63.0,55.0,52.0,77.0,67.0,42.0,...,10.0,10.0,10.0,18.0,4.0,0,0.0,clear,282.0,94063
2204,2015-08-31,78.0,69.0,60.0,58.0,57.0,54.0,84.0,67.0,50.0,...,10.0,10.0,9.0,18.0,9.0,0,1.0,clear,246.0,94107
2934,2015-08-31,82.0,70.0,59.0,66.0,59.0,54.0,82.0,64.0,48.0,...,20.0,17.0,10.0,17.0,8.0,0,0.0,clear,357.0,94301


From above, we see the weather_data contains data from 5 different weather stations.

In [70]:
# Keep weather data from only the first weather station.
weather_data = weather_data.groupby('date').first()
weather_data = weather_data.reset_index()

In [17]:
# 'T' in precipitation_inches represents 'Trace'. Replace all 'T' values with zeros.
weather_data = weather_data.replace({'precipitation_inches': {'T': float(0)}})

In [23]:
# Check if any null values are now present
weather_data.isnull().sum()

max_temperature_f                 0
mean_temperature_f                0
min_temperature_f                 0
max_dew_point_f                   0
mean_dew_point_f                  0
min_dew_point_f                   0
max_humidity                      0
mean_humidity                     0
min_humidity                      0
max_sea_level_pressure_inches     0
mean_sea_level_pressure_inches    0
min_sea_level_pressure_inches     0
max_visibility_miles              0
mean_visibility_miles             0
min_visibility_miles              0
max_wind_Speed_mph                0
mean_wind_speed_mph               0
precipitation_inches              0
cloud_cover                       0
events                            0
wind_dir_degrees                  0
zip_code                          0
dtype: int64

In [24]:
# Check that data is present for the entirety of the time period 2013-2015 (733 days)
weather_data.shape[0]

733

In [71]:
weather_data.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,2013-08-29,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,10.0,23.0,11.0,0,4.0,clear,286.0,94107
1,2013-08-30,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,10.0,7.0,29.0,13.0,0,2.0,clear,291.0,94107
2,2013-08-31,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,10.0,26.0,15.0,0,4.0,clear,284.0,94107
3,2013-09-01,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,10.0,25.0,13.0,0,4.0,clear,284.0,94107
4,2013-09-02,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,10.0,6.0,23.0,12.0,0,6.0,clear,277.0,94107


In [26]:
# Import the bike stations status data
stations= pd.read_csv('station.csv')
stations = stations.rename(columns = {'lat':'Latitude', 'long':'Longitude'})
stations.head()

,id,name,Latitude,Longitude,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [27]:
# Check the number of station names matches number od station id's
print(len(stations['id']))

70


In [28]:
print(len(stations['name']))

70


In [29]:
# Convert station_data into a GeoDataFrame.
stations = gpd.GeoDataFrame(stations, 
                                          geometry=gpd.points_from_xy(stations.Longitude, 
                                                                      stations.Latitude))
stations.crs = {'init': 'epsg:26917'}

C:\Users\Emanuele\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [30]:
stations.head()

,id,name,Latitude,Longitude,dock_count,city,installation_date,geometry
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,POINT (-121.902 37.330)
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013,POINT (-121.889 37.331)
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013,POINT (-121.895 37.334)
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013,POINT (-121.893 37.331)
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013,POINT (-121.894 37.337)


In [28]:
stations.shape

(70, 8)

In [33]:
# Check for the presence of null values
stations.isnull().sum()

id                   0
name                 0
Latitude             0
Longitude            0
dock_count           0
city                 0
installation_date    0
geometry             0
dtype: int64

In [34]:
# Import transit stops data for SF
transit_stops = pd.read_csv('Muni_Stops.csv')

In [35]:
transit_stops.head()

,OBJECTID,STOPNAME,TRAPEZESTOPABBR,RUCUSSTOPABBR,STOPID,LATITUDE,LONGITUDE,ACCESSIBILITYMASK,ATSTREET,ONSTREET,...,INSERT_TIMESTAMP,SDE_ID,SIGNUPID,SUPERVISOR_DISTRICT,shape,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,36073,Powell St&Francisco St SE-NS/BZ,POWLFNC1,POWLFNCO,6056,37.804810,-122.411650,0.0,FRANCISCO ST,POWELL ST,...,20220727151100,14257412,134,NaN,POINT (-122.41165 37.80481),106.0,106.0,6.0,3.0,23.0
1,33704,Chestnut St&Laguna St SW-NS/BZ,CHESLGN1,CHESLGNA,3948,37.801370,-122.431406,0.0,LAGUNA ST,CHESTNUT ST,...,20220727151100,14253217,134,NaN,POINT (-122.431404 37.80137),17.0,17.0,4.0,6.0,13.0
2,33872,Geary Blvd&Fillmore St NW-FS/BZ,GEARFIL0,GEARFILL,4295,37.784391,-122.433050,0.0,AVERY ST,GEARY BLVD,...,20220727151100,14253429,134,NaN,POINT (-122.43305 37.78439),103.0,103.0,4.0,11.0,39.0
3,33441,3rd St&Folsom St N-FS/BZ,.3STFOL0,3STFOLS,3124,37.784204,-122.399326,0.0,CLEMENTINA ST,03RD ST,...,20220727151100,14252566,134,NaN,POINT (-122.39932 37.784203),32.0,32.0,1.0,10.0,8.0
4,35962,Presidio Ave&Sutter St NE-FS/PS,PRESSUT1,PRESSUTT,6098,37.784535,-122.446197,0.0,SUTTER ST,PRESIDIO AVE,...,20220727151100,14258040,134,NaN,POINT (-122.4462 37.784534),103.0,103.0,8.0,6.0,31.0


In [36]:
# Keep only the stop address, latitude and longitude features
transit_stops = transit_stops[['STOPNAME', 'LATITUDE', 'LONGITUDE']]
transit_stops = transit_stops.rename(columns = {'STOPNAME':'Address', 'LATITUDE':'Latitude', 'LONGITUDE':'Longitude'})

In [37]:
transit_stops = gpd.GeoDataFrame(transit_stops, 
                                          geometry=gpd.points_from_xy(transit_stops.Longitude, 
                                                                      transit_stops.Latitude))
transit_stops.crs = {'init': 'epsg:26917'}

C:\Users\Emanuele\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [38]:
transit_stops.head()

,Address,Latitude,Longitude,geometry
0,Powell St&Francisco St SE-NS/BZ,37.804810,-122.411650,POINT (-122.412 37.805)
1,Chestnut St&Laguna St SW-NS/BZ,37.801370,-122.431406,POINT (-122.431 37.801)
2,Geary Blvd&Fillmore St NW-FS/BZ,37.784391,-122.433050,POINT (-122.433 37.784)
3,3rd St&Folsom St N-FS/BZ,37.784204,-122.399326,POINT (-122.399 37.784)
4,Presidio Ave&Sutter St NE-FS/PS,37.784535,-122.446197,POINT (-122.446 37.785)


In [39]:
# Import additional transit stops data to supplement the previous one
transit2 = gpd.read_file('Bus_Routes_and_Stops_January_2020.geojson')
transit2.head()

,FID,LineDirId,StopName,RTID,Routes,geometry
0,0,211870,Santa Clara Transit Center,60001,22,MULTIPOINT (-121.93667 37.35305)
1,1,212170,San Jose Airport Terminal A,60008,"60, 60",MULTIPOINT (-121.92760 37.36842)
2,2,212171,San Jose Airport Terminal B,60011,"60, 60",MULTIPOINT (-121.92307 37.36445)
3,3,212171,1st & Metro @ Metro Station,60014,60,MULTIPOINT (-121.91588 37.36977)
4,4,211870,El Camino & Lafayette,60020,"22, 60, 59",MULTIPOINT (-121.94514 37.35506)


In [40]:
# Extract the Latitude and Longitude from the geometry column
transit2["geo_string"] = str(transit2['geometry'])
transit2["geo_string"] = transit2["geo_string"].apply(lambda x: (x.split(')')[0]).split('(')[1].split(' '))

transit2['Latitude'] = transit2['geo_string'].apply(lambda x: float(x[1]))
transit2['Longitude'] = transit2['geo_string'].apply(lambda x: float(x[0]))

In [41]:
# Keep only the stop address, latitude and longitude features
transit2 = transit2[['StopName', 'Latitude', 'Longitude']]
transit2 = transit2.rename(columns={'StopName':'Address'})

In [42]:
transit2 = gpd.GeoDataFrame(transit2, 
                                          geometry=gpd.points_from_xy(transit2.Longitude, 
                                                                      transit2.Latitude))
transit2.crs = {'init': 'epsg:26917'}

C:\Users\Emanuele\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [43]:
transit2.head()

,Address,Latitude,Longitude,geometry
0,Santa Clara Transit Center,37.35305,-121.93667,POINT (-121.937 37.353)
1,San Jose Airport Terminal A,37.35305,-121.93667,POINT (-121.937 37.353)
2,San Jose Airport Terminal B,37.35305,-121.93667,POINT (-121.937 37.353)
3,1st & Metro @ Metro Station,37.35305,-121.93667,POINT (-121.937 37.353)
4,El Camino & Lafayette,37.35305,-121.93667,POINT (-121.937 37.353)


In [44]:
# Convert transit2 and transit_stops to csv dataframes
transit2 = pd.DataFrame(transit2)
transit_stops = pd.DataFrame(transit_stops)

In [45]:
# Concatenate transit2 to transit_stops
transit_stops = pd.concat([transit2, transit_stops], ignore_index=True)

In [46]:
transit_stops.head()

,Address,Latitude,Longitude,geometry
0,Santa Clara Transit Center,37.35305,-121.93667,POINT (-121.937 37.353)
1,San Jose Airport Terminal A,37.35305,-121.93667,POINT (-121.937 37.353)
2,San Jose Airport Terminal B,37.35305,-121.93667,POINT (-121.937 37.353)
3,1st & Metro @ Metro Station,37.35305,-121.93667,POINT (-121.937 37.353)
4,El Camino & Lafayette,37.35305,-121.93667,POINT (-121.937 37.353)


In [49]:
transit_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Address    6819 non-null   object  
 1   Latitude   6819 non-null   float64 
 2   Longitude  6819 non-null   float64 
 3   geometry   6819 non-null   geometry
dtypes: float64(2), geometry(1), object(1)
memory usage: 213.2+ KB


In [50]:
# Perform a proximity analysis.
# Find the shortest distance  between a bike station and a transit stop.

In [51]:
''' We’ll use the Haversine (or Great Circle) distance formula, 
    which takes the latitude and longitude of two points, adjusts 
    for Earth’s curvature, and calculates the straight-line distance between them. '''

def haversine(lat1, lon1, lat2, lon2):
    """Defines a basic Haversine distance formula."""
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [52]:
# Create a function that vectorizes the haversine function.

def vectorized_array_method(bike_stations, transit_stops):
    
    # make pairs of stations (subway)
    stn_latlong  = list(zip(transit_stops['Latitude'].to_numpy(), transit_stops['Longitude'].to_numpy()))
    
    # make pairs of bike stations
    bike_latlong = list(zip(bike_stations['Latitude'].to_numpy(), bike_stations['Longitude'].to_numpy()))
    
    # for every bike station, find the closest subway station using the Haversine function 
    bike_dist = [
        min([ haversine(stn_lat, stn_lon, lat2, lon2) for (stn_lat, stn_lon) in stn_latlong ]) 
        for (lat2, lon2) in bike_latlong
    ]
    
    bike_stations["distance_to_nearest_transit_Miles"] = bike_dist

#     bike_stations[distance] = haversine(lat1 = ... #we should have the closest subway station here , 
#                                       lon1 = ... #we should have the closest subway station here   , 
#                                       lat2=bike_stations['Latitude'].to_numpy(), 
#                                       lon2=bike_stations['Longitude'].to_numpy())
    
    return bike_stations

In [53]:
# Create a dataframe containg, for each bike station, the distance to the nearest transit stop
closest_dist_df = vectorized_array_method(stations, transit_stops)

In [54]:
closest_dist_df.head()

,id,name,Latitude,Longitude,dock_count,city,installation_date,geometry,distance_to_nearest_transit_Miles
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,POINT (-121.902 37.330),2.503852
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013,POINT (-121.889 37.331),3.041251
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013,POINT (-121.895 37.334),2.645639
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013,POINT (-121.893 37.331),2.817329
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013,POINT (-121.894 37.337),2.597729


In [55]:
closest_dist_df = closest_dist_df.rename(columns={'name':'bike_station_name', 'id':'station_id'})
closest_dist_final = closest_dist_df[['station_id', 'bike_station_name', 'Latitude', 'Longitude', 'distance_to_nearest_transit_Miles']]

In [56]:
closest_dist_final.sort_values(by=['distance_to_nearest_transit_Miles'])

,station_id,bike_station_name,Latitude,Longitude,distance_to_nearest_transit_Miles
57,69,San Francisco Caltrain 2 (330 Townsend),37.776600,-122.395470,0.001692
59,71,Powell at Post (Union Square),37.788446,-122.408499,0.003746
62,74,Steuart at Market,37.794139,-122.394434,0.009413
64,76,Market at 4th,37.786305,-122.404966,0.012113
58,70,San Francisco Caltrain (Townsend at 4th),37.776617,-122.395260,0.012846
...,...,...,...,...,...
16,23,San Mateo County Center,37.487616,-122.229951,17.743608
15,22,Redwood City Caltrain Station,37.486078,-122.232089,17.776614
19,26,Redwood City Medical Center,37.487682,-122.223492,17.921529
17,24,Redwood City Public Library,37.484219,-122.227424,18.016174


In [58]:
print(closest_dist_final['distance_to_nearest_transit_Miles'].min())
print(closest_dist_final['distance_to_nearest_transit_Miles'].mean())
print(closest_dist_final['distance_to_nearest_transit_Miles'].max())

0.0016918478705577407
4.267706403644955
18.177371660108626


In [59]:
# Make sure that the number of bike station names = number of station id's
bike_station_names = closest_dist_final['bike_station_name'].unique().tolist()
len(bike_station_names)

70

In [60]:
bike_station_ids = closest_dist_final['station_id'].unique().tolist()
len(bike_station_ids)

70

In [61]:
# Import the bike stations status data
status_data = pd.read_csv('status.csv')

In [76]:
# Add distance_to_nearest_subway to the bike stations in status_data.
closest_dist_final.rename(columns ={'id':'station_id'}, inplace=True)
status_data = status_data.merge(closest_dist_final, on='station_id', how = 'left')
status_data.head()

C:\Users\Emanuele\AppData\Local\Temp\ipykernel_1420\389356350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closest_dist_final.rename(columns ={'id':'station_id'}, inplace=True)


,station_id,bikes_available,docks_available,time,bike_station_name,Latitude,Longitude,distance_to_nearest_transit_Miles
0,2,2,25,2013-08-29 12:06:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
1,2,2,25,2013-08-29 12:07:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
2,2,2,25,2013-08-29 12:08:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
3,2,2,25,2013-08-29 12:09:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
4,2,2,25,2013-08-29 12:10:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852


In [77]:
# Convert status_data to a dateTime dataFrame
status_data['time'] = pd.to_datetime(status_data['time'])

In [78]:
# Round status data to the nearest minute.
status_data['time'] = status_data['time'].dt.round('min')

In [79]:
# Rename status_data to complete_status_data 
complete_status_data = status_data

In [80]:
complete_status_data.head()

,station_id,bikes_available,docks_available,time,bike_station_name,Latitude,Longitude,distance_to_nearest_transit_Miles
0,2,2,25,2013-08-29 12:06:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
1,2,2,25,2013-08-29 12:07:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
2,2,2,25,2013-08-29 12:08:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
3,2,2,25,2013-08-29 12:09:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852
4,2,2,25,2013-08-29 12:10:00,San Jose Diridon Caltrain Station,37.329732,-121.901782,2.503852


In [81]:
# Aggregate complete_status_data to an hourly basis, taking the hourly mean for continuous features.
hourly_status_data = complete_status_data.groupby(['bike_station_name', complete_status_data['time'].dt.floor('H')]).agg(
    bikes_available = ('bikes_available', 'mean'),
    docks_available = ('docks_available', 'mean'),
    distance_to_nearest_transit_Miles = ('distance_to_nearest_transit_Miles', lambda x: x.iloc[0] ),
    Latitude = ('Latitude', lambda x: x.iloc[0] ),
    Longitude = ('Longitude', lambda x: x.iloc[0] ))
# View DataFrame
hourly_status_data.head(50)

bikes_available  docks_available  \
bike_station_name time                                                    
2nd at Folsom     2013-08-29 12:00:00         9.500000         9.500000   
                  2013-08-29 13:00:00         9.264151         9.735849   
                  2013-08-29 14:00:00         9.928571         9.071429   
                  2013-08-29 15:00:00        10.055556         8.944444   
                  2013-08-29 16:00:00        10.000000         9.000000   
                  2013-08-29 17:00:00         9.574074         9.425926   
                  2013-08-29 18:00:00        10.555556         8.444444   
                  2013-08-29 19:00:00         5.836364        13.163636   
                  2013-08-29 20:00:00         4.019231        14.980769   
                  2013-08-29 21:00:00         4.000000        15.000000   
                  2013-08-29 22:00:00         4.000000        15.000000   
                  2013-08-29 23:00:00         4.000000        15.000000   
                  2013-08-30 00:00:00         4.000000        15.000000   
                  2013-08-30 01:00:00         4.000000        15.000000   
                  2013-08-30 02:00:00         4.000000        15.000000   
                  2013-08-30 03:00:00         4.000000        15.000000   
                  2013-08-30 04:00:00         4.000000        15.000000   
                  2013-08-30 05:00:00         4.000000        15.000000   
                  2013-08-30 06:00:00         4.000000        15.000000   
                  2013-08-30 07:00:00         4.000000        15.000000   
                  2013-08-30 08:00:00         4.000000        15.000000   
                  2013-08-30 09:00:00         4.490196        14.509804   
                  2013-08-30 10:00:00         2.859649        16.140351   
                  2013-08-30 11:00:00         2.296296        16.703704   
                  2013-08-30 12:00:00         3.314815        15.685185   
                  2013-08-30 13:00:00         3.724138        15.275862   
                  2013-08-30 14:00:00         4.315789        14.684211   
                  2013-08-30 15:00:00         5.730769        13.269231   
                  2013-08-30 16:00:00         5.545455        13.454545   
                  2013-08-30 17:00:00         5.000000        14.000000   
                  2013-08-30 18:00:00         3.415094        15.584906   
                  2013-08-30 19:00:00         2.000000        17.000000   
                  2013-08-30 20:00:00         2.000000        17.000000   
                  2013-08-30 21:00:00         2.618182        16.381818   
                  2013-08-30 22:00:00         3.000000        16.000000   
                  2013-08-30 23:00:00         3.000000        16.000000   
                  2013-08-31 00:00:00         3.000000        16.000000   
                  2013-08-31 01:00:00         3.000000        16.000000   
                  2013-08-31 02:00:00         3.000000        16.000000   
                  2013-08-31 03:00:00         3.000000        16.000000   
                  2013-08-31 04:00:00         3.000000        16.000000   
                  2013-08-31 05:00:00         3.000000        16.000000   
                  2013-08-31 06:00:00         3.000000        16.000000   
                  2013-08-31 07:00:00         3.000000        16.000000   
                  2013-08-31 08:00:00         3.000000        16.000000   
                  2013-08-31 09:00:00         2.019231        16.980769   
                  2013-08-31 10:00:00         2.875000        16.125000   
                  2013-08-31 11:00:00         2.000000        17.000000   
                  2013-08-31 12:00:00         2.000000        17.000000   
                  2013-08-31 13:00:00         1.017241        17.982759   

                                       distance_to_nearest_transit_Miles  \
bike_station_name time                                        

In [82]:
# Merge hourly_status_data and the weather_data
hourly_status_data = hourly_status_data.reset_index()
hourly_status_data["merge_time"] = hourly_status_data["time"].dt.date
hourly_status_data['merge_time'] = pd.to_datetime(hourly_status_data['merge_time'])
weather_data.rename(columns = {'date':'merge_time'}, inplace = True)

inner2 = hourly_status_data.merge(weather_data, left_on = 'merge_time', right_on = 'merge_time', how = 'right')
inner2.head()

In [85]:
inner2.isnull().sum()

bike_station_name                    0
time                                 0
bikes_available                      0
docks_available                      0
distance_to_nearest_transit_Miles    0
Latitude                             0
Longitude                            0
merge_time                           0
max_temperature_f                    0
mean_temperature_f                   0
min_temperature_f                    0
max_dew_point_f                      0
mean_dew_point_f                     0
min_dew_point_f                      0
max_humidity                         0
mean_humidity                        0
min_humidity                         0
max_sea_level_pressure_inches        0
mean_sea_level_pressure_inches       0
min_sea_level_pressure_inches        0
max_visibility_miles                 0
mean_visibility_miles                0
min_visibility_miles                 0
max_wind_Speed_mph                   0
mean_wind_speed_mph                  0
precipitation_inches     

In [86]:
# Create a new column representing the percentage of available bikes at each station on an hourly basis.
inner2['percentage'] = inner2['docks_available']/(inner2['docks_available']+inner2['bikes_available'])
inner2['per'] = inner2['docks_available']/(inner2['docks_available']+inner2['bikes_available'])

# Create a new column named '50%', representing our target variable. This column assigns a number of 1 if the percentage of 1
# bikes available is greater than 50%, and it assigns a value of 0 if % of bikes available is less than 50%
inner2['per'] = np.where((inner2.per >= 0.5), 1, inner2.per)
inner2['per'] = np.where((inner2.per < 0.5), 0, inner2.per)
inner2.rename(columns ={'per':'50%'}, inplace=True)

In [87]:
inner2.sort_values(by=['bike_station_name','time'])

,bike_station_name,time,bikes_available,docks_available,distance_to_nearest_transit_Miles,Latitude,Longitude,merge_time,max_temperature_f,mean_temperature_f,...,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code,percentage,50%
0,2nd at Folsom,2013-08-29 12:00:00,9.500000,9.500000,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,...,10.0,23.0,11.0,0,4.0,clear,286.0,94107,0.500000,1.0
1,2nd at Folsom,2013-08-29 13:00:00,9.264151,9.735849,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,...,10.0,23.0,11.0,0,4.0,clear,286.0,94107,0.512413,1.0
2,2nd at Folsom,2013-08-29 14:00:00,9.928571,9.071429,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,...,10.0,23.0,11.0,0,4.0,clear,286.0,94107,0.477444,0.0
3,2nd at Folsom,2013-08-29 15:00:00,10.055556,8.944444,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,...,10.0,23.0,11.0,0,4.0,clear,286.0,94107,0.470760,0.0
4,2nd at Folsom,2013-08-29 16:00:00,10.000000,9.000000,0.031746,37.785299,-122.396236,2013-08-29,74.0,68.0,...,10.0,23.0,11.0,0,4.0,clear,286.0,94107,0.473684,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204829,Yerba Buena Center of the Arts (3rd @ Howard),2015-08-31 19:00:00,7.000000,11.000000,0.103277,37.784878,-122.401014,2015-08-31,78.0,69.0,...,9.0,18.0,9.0,0,1.0,clear,246.0,94107,0.611111,1.0
1204830,Yerba Buena Center of the Arts (3rd @ Howard),2015-08-31 20:00:00,5.750000,12.666667,0.103277,37.784878,-122.401014,2015-08-31,78.0,69.0,...,9.0,18.0,9.0,0,1.0,clear,246.0,94107,0.687783,1.0
1204831,Yerba Buena Center of the Arts (3rd @ Howard),2015-08-31 21:00:00,5.000000,14.000000,0.103277,37.784878,-122.401014,2015-08-31,78.0,69.0,...,9.0,18.0,9.0,0,1.0,clear,246.0,94107,0.736842,1.0
1204832,Yerba Buena Center of the Arts (3rd @ Howard),2015-08-31 22:00:00,5.000000,14.000000,0.103277,37.784878,-122.401014,2015-08-31,78.0,69.0,...,9.0,18.0,9.0,0,1.0,clear,246.0,94107,0.736842,1.0


In [88]:
# Remove the outliers. We consider outliers duration values less than Q1 - 1.5 * IQR and greater than Q3 + 1.5 * IQR.
'''
Q1: The first quartile (.quantile(0.25))
Q3: The third quartile (.quantile(0.75))
IQR: The first quartil (Q3 - Q1)

'''
Q1 = inner2['distance_to_nearest_transit_Miles'].quantile(0.25)
Q3 = inner2['distance_to_nearest_transit_Miles'].quantile(0.75)
IQR = Q3-Q1
inner2 = inner2[(inner2['distance_to_nearest_transit_Miles'] >= (Q1-1.5*IQR)) & (inner2['distance_to_nearest_transit_Miles'] <= (Q3+1.5*IQR))]

In [82]:
# Export the final cleaned dataframe to a .csv file.
inner2.to_csv('clean_weather_station.csv', index = False)


In [84]:
transit_stops.to_csv('transit_stops.csv', index = False)

In [83]:
inner2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1204834 entries, 0 to 1204833
Data columns (total 33 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   bike_station_name                  1204834 non-null  object        
 1   time                               1204834 non-null  datetime64[ns]
 2   bikes_available                    1204834 non-null  float64       
 3   docks_available                    1204834 non-null  float64       
 4   distance_to_nearest_transit_Miles  1204834 non-null  float64       
 5   Latitude                           1204834 non-null  float64       
 6   Longitude                          1204834 non-null  float64       
 7   merge_time                         1204834 non-null  datetime64[ns]
 8   index                              1204834 non-null  int64         
 9   max_temperature_f                  1204834 non-null  float64       
 10  mean_t

In [8]:
inner2 = inner2.replace({'precipitation_inches': {'T': float(0)}})